In [102]:
# Sample code on old dataset
import numpy as np
import csv
import os
import pandas as pd
import sys
sys.path.append('..')

cwd = os.getcwd()
training_dataframe = pd.read_csv('data_train.csv')
training_dataframe
#training_matrix = training_dataframe.to_numpy()
#training_matrix

,row,col,data
0,0,3568,1.0
1,0,3827,1.0
2,0,4844,1.0
3,0,5734,1.0
4,0,6518,1.0
...,...,...,...
398631,30910,18176,1.0
398632,30910,18185,1.0
398633,30910,18248,1.0
398634,30910,18349,1.0


In [103]:
# alcuni utenti non hanno interaction e alcuni items non hanno interactions 
unique_users = training_dataframe["row"].unique()
unique_items = training_dataframe["col"].unique()
print(training_dataframe["col"].max())
print(training_dataframe["col"].min())

unique_data = training_dataframe["data"].unique()

print('Number of unique users:',len(unique_users))
print('Number of unique items:',len(unique_items))
print('Number of unique data:',len(unique_data))

matrix_dimension = len(unique_users) * len(unique_items)
percentage_of_data = (len(training_dataframe.index)/matrix_dimension)*100
print("Percentage of non_null cells:",percentage_of_data)

18494
0
Number of unique users: 27255
Number of unique items: 15277
Number of unique data: 1
Percentage of non_null cells: 0.09573974168719476


In [104]:
#converting data into sparse matrix
#NB: sps.coo_matrix(...) generates a NxM matrix where N is the max index in row and M is the max index in col.

import scipy.sparse as sps
row = training_dataframe["row"].to_numpy()
col = training_dataframe["col"].to_numpy()
data = training_dataframe["data"].to_numpy()

URM_all = sps.coo_matrix((data, (row,col))) #smart way to couple indexes and data. It's not fast btw. Convert it into CSR format
URM_all.tocsr()

<30911x18495 sparse matrix of type '<class 'numpy.float64'>'
	with 398636 stored elements in Compressed Sparse Row format>

In [105]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

#il primo lo uso per parameter tuning, il secondo per testare alla fine
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10]) #evaluator sulla validation
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10]) #evaluator sul test

In [106]:
from Base.NonPersonalizedRecommender import TopPop
user_id = np.unique(row)[0:10]
recommender_top_popular = TopPop(URM_train)
recommender_top_popular.fit()
recommendations = recommender_top_popular.recommend(user_id,cutoff = 10)

result_dict, _ = evaluator_test.evaluateRecommender(recommender_top_popular)
result_dict

TopPopRecommender: URM Detected 5056 (16.36 %) cold users.
TopPopRecommender: URM Detected 4248 (22.97 %) cold items.
EvaluatorHoldout: Processed 20451 ( 100.00% ) in 13.18 sec. Users per second: 1551


{10: {'ROC_AUC': 0.03820288797148119,
  'PRECISION': 0.00793115251087945,
  'PRECISION_RECALL_MIN_DEN': 0.022980926923276884,
  'RECALL': 0.022430442474506877,
  'MAP': 0.007064723206175406,
  'MRR': 0.02366201676015375,
  'NDCG': 0.014506285521237082,
  'F1': 0.011718703199713252,
  'HIT_RATE': 0.07931152510879663,
  'ARHR': 0.024391363638833187,
  'NOVELTY': 0.004839337005584089,
  'AVERAGE_POPULARITY': 0.8815289454874433,
  'DIVERSITY_MEAN_INTER_LIST': 0.04150008410585795,
  'DIVERSITY_HERFINDAHL': 0.9041498054861121,
  'COVERAGE_ITEM': 0.0011354420113544202,
  'COVERAGE_ITEM_CORRECT': 0.0008650986753176534,
  'COVERAGE_USER': 0.6616091359063117,
  'COVERAGE_USER_CORRECT': 0.049464591892853675,
  'DIVERSITY_GINI': 0.0006085057050027081,
  'SHANNON_ENTROPY': 3.423593722586453}}

In [107]:
test_users = pd.read_csv('data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,4
4,5
...,...
28101,30906
28102,30907
28103,30908
28104,30909


In [113]:
user_id = test_users['user_id']
recommendations = recommender_top_popular.recommend(user_id,cutoff = 10)

In [119]:
recommendations[0]

array([17955,  8638,  5113,  8982,  4361,  4657, 10227,   197, 12469,
       10466])

In [110]:
sample_output = pd.read_csv('baseline_sample_submission.csv')
sample_output

,user_id,item_list
0,0,0 1 2 3 4 5 6 7 8 9
1,1,0 1 2 3 4 5 6 7 8 9
2,2,0 1 2 3 4 5 6 7 8 9
3,4,0 1 2 3 4 5 6 7 8 9
4,5,0 1 2 3 4 5 6 7 8 9
...,...,...
28101,30906,0 1 2 3 4 5 6 7 8 9
28102,30907,0 1 2 3 4 5 6 7 8 9
28103,30908,0 1 2 3 4 5 6 7 8 9
28104,30909,0 1 2 3 4 5 6 7 8 9


In [115]:
for index in range(len(recommendations)):
    #print(element)
    recommendations[index]=np.array(recommendations[index])
    #print(type(element))
print(len(recommendations))

28106


In [121]:
test_users['item_list']= recommendations
#test_users['item_list'] =  test_users['item_list'].apply(lambda x: x.replace('[','').replace(']','')) 
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

#convert the string columns to int
#test_users['item_list'] = test_users['item_list'].astype(int)
test_users
test_users.to_csv('submission.csv', index=False)
